In [105]:
# -*- coding: utf-8 -*-
import sys
import codecs
import json

In [106]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
from http.server import BaseHTTPRequestHandler, HTTPServer
import urllib.parse as urlparse
import pandas as pd
import os

In [107]:
link = "https://batdongsan.vn/"
t = "/ban-nha/p"
r = requests.get(link+t[0])
soup = BeautifulSoup(r.text, 'html.parser')




In [108]:
def find_link(soup, link):
    l = []
    div_items = soup.find_all("div", attrs={"class" : "item"})
    for div_item in div_items:
        a_links = div_item.find_all('a',href=True)
        for a_link in a_links:
            l.append(a_link['href'])
    return l

# for value in find_link(soup, link):
#     print(value)

In [109]:
def find_data(soup):
    data = {}

    # Tiêu đề
    titles = soup.find_all('div', {"class": "name"})
    data['title'] = [title.find('a').text.strip() for title in titles]

    # Giá
    prices = soup.find_all('span', {"class": "price"})
    data['price'] = [price.text.strip() for price in prices]

    # Diện tích
    acreages = soup.find_all('span', {"class": "acreage"})
    data['acreage'] = [acreage.text.strip() for acreage in acreages]

    # Mô tả
    descriptions = soup.find_all('div', {"class": "sapo"})
    data['description'] = [description.find('p').text.strip() for description in descriptions]

    return data

def process_property_elements(soup):
    data_list = []

    # Tìm tất cả các phần tử HTML chứa thông tin
    property_elements = soup.find_all('div', {"class": "item"})

    for property_element in property_elements:
        # Gọi hàm find_data cho mỗi phần tử và thêm kết quả vào danh sách
        data_list.append(find_data(property_element))

    return data_list

def scrape_and_save(link, base_link, t, max_pages):
    all_data = []

    for i in range(1, max_pages + 1):
        try:
            r = requests.get(link + t + str(i))
            r.raise_for_status()  # Raise HTTPError for bad requests
            soup = BeautifulSoup(r.text, 'html.parser')
            
            property_data = process_property_elements(soup)
            all_data.extend(property_data)

        except requests.RequestException as e:
            print(f"Error fetching page {i}: {e}")

    with open('bat_dong_san.json', 'w') as file:
        json.dump(all_data, file)

    print("Done")

In [110]:
max_pages = 50

scrape_and_save(link, link, t, max_pages)

Done


In [111]:
base_url = "https://batdongsan.com.vn/nha-dat-ban"


def extract_data_from_product(product):
    def extract_element_text(parent, selector):
        element = parent.select_one(selector)
        return element.text.strip() if element else None

    def extract_image_src(parent, selector):
        element = parent.select_one(selector)
        return element["src"] if element else None

    data = {
        "prid": product.get("prid"),
        "title": product.select_one("a.js__product-link-for-product-id").get("title") if product.select_one("a.js__product-link-for-product-id") else None,
        "location": extract_element_text(product, "div.re__card-location"),
        "time_published": product.select_one("span.re__card-published-info-published-at").get("aria-label") if product.select_one("span.re__card-published-info-published-at") else None,
        "price": extract_element_text(product, "span.re__card-config-price"),
        "area": extract_element_text(product, "span.re__card-config-area"),
        "price_per_m2": extract_element_text(product, "span.re__card-config-price_per_m2"),
        "bedrooms": extract_element_text(product, "span.re__card-config-bedroom"),
        "bathrooms": extract_element_text(product, "span.re__card-config-toilet"),
        "agent_name": extract_element_text(product, "div.re__card-published-info-agent-profile-name"),
        "product_url": base_url + product.select_one("a.js__product-link-for-product-id").get("href") if product.select_one("a.js__product-link-for-product-id") else None,
        "agent_avatar": extract_image_src(product, "div.re__card-published-info-agent-profile-avatar img") if extract_image_src(product, "div.re__card-published-info-agent-profile-avatar img") else None,
        "main_image": product.select_one("div.re__img-child img[data-img]")["data-img"] if product.select_one("div.re__img-child img[data-img]") else None,
        "other_images": [img["data-src"] for img in product.select("div.re__img-child img[data-src]")],
        "description": extract_element_text(product, "div.re__card-description.js__card-description"),
    }

    return data


def crawl_with_zenrows(url):
    all_products = []
    with tqdm(total=50) as pbar:
        for page_number in range(1, 51):
            params = {
                'url': f"{url}/p{page_number}",
                'apikey': 'fbda2bc1d8a35edf5bfdf3f2dad6b66de5c73e50',
                'premium_proxy': 'true',
            }
            response = requests.get(
                'https://api.zenrows.com/v1/', params=params)

            if response.status_code == 200:
                html_content = response.text
                soup = BeautifulSoup(html_content, "html.parser")
                product_elements = soup.find_all("div", {"class": "js__card"})

                for product in product_elements:
                    all_products.append(extract_data_from_product(product))
            else:
                print(f"Failed to fetch data from page {page_number}")
                break

            pbar.update(1)

    return all_products


all_products = crawl_with_zenrows(base_url)

if all_products:
    df = pd.DataFrame(all_products)

    df.to_csv("products_data.csv", index=False)
    print("Data from 50 pages saved to 'products_data.csv'")
    df.to_json("products_data.json", orient="records")
    print("Data from 50 pages saved to 'products_data.json'")
else:
    print("No data to save.")

  0%|          | 0/50 [00:01<?, ?it/s]

Failed to fetch data from page 1
No data to save.
